In [76]:
from datasets import load_dataset
from datasets import Dataset, DatasetDict
import torch


import os
import random
import pandas as pd
import numpy as np

In [77]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(42)

In [78]:
# seara/ru_go_emotions
goemotions = load_dataset("seara/ru_go_emotions", "raw")

In [79]:
# djacon/ru-izard-emotions
izard = load_dataset("djacon/ru-izard-emotions")

In [80]:
izard

DatasetDict({
    train: Dataset({
        features: ['text', 'neutral', 'joy', 'sadness', 'anger', 'enthusiasm', 'surprise', 'disgust', 'fear', 'guilt', 'shame'],
        num_rows: 20162
    })
    validation: Dataset({
        features: ['text', 'neutral', 'joy', 'sadness', 'anger', 'enthusiasm', 'surprise', 'disgust', 'fear', 'guilt', 'shame'],
        num_rows: 2240
    })
    test: Dataset({
        features: ['text', 'neutral', 'joy', 'sadness', 'anger', 'enthusiasm', 'surprise', 'disgust', 'fear', 'guilt', 'shame'],
        num_rows: 2489
    })
})

In [81]:
goemotions

DatasetDict({
    train: Dataset({
        features: ['ru_text', 'text', 'id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear', 'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy', 'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief', 'remorse', 'sadness', 'surprise', 'neutral'],
        num_rows: 211225
    })
})

In [82]:
# Создадим пустой датасет для объединения всех датасетов с эмоциями
labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
emo_labels = ["text"] + labels
emotions_df = pd.DataFrame(columns=emo_labels)

In [83]:
# Приведем датасет goemotions к криптонианскому виду
goemotions_df_train = goemotions["train"].to_pandas().rename(columns={"ru_text": "text", "text": "en_text"})
goemotions_df_train.columns
# Оставим только поля text и эмоции из списка labels

Index(['text', 'en_text', 'id', 'author', 'subreddit', 'link_id', 'parent_id',
       'created_utc', 'rater_id', 'example_very_unclear', 'admiration',
       'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion',
       'curiosity', 'desire', 'disappointment', 'disapproval', 'disgust',
       'embarrassment', 'excitement', 'fear', 'gratitude', 'grief', 'joy',
       'love', 'nervousness', 'optimism', 'pride', 'realization', 'relief',
       'remorse', 'sadness', 'surprise', 'neutral'],
      dtype='object')

In [84]:
def add_crypto_labels(df):
    # Проверим, есть ли все необходимые колонки представлены в датасете
    missing_columns = [col for col in emo_labels if col not in df.columns]

    # Добавим недостающие колонки и заполним их нулями
    for col in missing_columns:
        df[col] = 0

    return df

goemotions_df_train = add_crypto_labels(goemotions_df_train)

# Оставим только необходимые эмоции
goemotions_df_train = goemotions_df_train[emo_labels]
print(goemotions_df_train.shape)

# Удалим записи, для которых все эмоции равны 0
goemotions_df_train = goemotions_df_train[goemotions_df_train[labels].sum(axis=1) > 0]
print(goemotions_df_train.shape)

(211225, 8)
(90698, 8)


In [85]:
goemotions_df_train

,text,anger,disgust,fear,joy,sadness,surprise,neutral
0,Эта игра причинила мне боль.,0,0,0,0,1,0,0
2,"Ты правильно поступаешь, если тебе все равно, ...",0,0,0,0,0,0,1
4,"[ИМЯ] не было рядом с ними, он был рядом с «Со...",0,0,0,0,0,0,1
10,"Да, и теперь, когда вы упомянули об этом, я ду...",0,0,0,0,0,0,1
12,НО ЭТО ОНА! /с,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...
211212,"Какое великолепное фото, и вы оба выглядите та...",0,0,0,1,0,0,0
211213,"Ух ты, она теперь хедлайнер двух шоу",0,0,0,0,0,0,1
211215,Ютуб — это моя Википедия. Видео на любой интер...,0,0,0,0,0,0,1
211219,"Что ж, я рад, что ты теперь свободен от всего ...",0,0,0,1,0,0,0


In [86]:
# Посмотрим на распределение эмоций в goemotions_df_train
goemotions_df_train[labels].sum()

anger        8084
disgust      5301
fear         3197
joy          7983
sadness      6758
surprise     5514
neutral     55298
dtype: int64

In [87]:
# Аналогично приведем датасет izard к криптонианскому виду
izard_df_train = izard["train"].to_pandas()
izard_df_test = izard["test"].to_pandas()
izard_df_val = izard["validation"].to_pandas()

In [88]:
izard_df_train = add_crypto_labels(izard_df_train)
izard_df_test = add_crypto_labels(izard_df_test)
izard_df_val = add_crypto_labels(izard_df_val)

In [89]:
izard_df_train = izard_df_train[emo_labels]
izard_df_test = izard_df_test[emo_labels]
izard_df_val = izard_df_val[emo_labels]

In [90]:
print(izard_df_train.shape)
izard_df_train = izard_df_train[izard_df_train[labels].sum(axis=1) > 0]
print(izard_df_train.shape)

print(izard_df_test.shape)
izard_df_test = izard_df_test[izard_df_test[labels].sum(axis=1) > 0]
print(izard_df_test.shape)

print(izard_df_val.shape)
izard_df_val = izard_df_val[izard_df_val[labels].sum(axis=1) > 0]
print(izard_df_val.shape)

(20162, 8)
(19186, 8)
(2489, 8)
(2344, 8)
(2240, 8)
(2156, 8)


In [91]:
# Посмотри на распределение эмоций в izard_df_train
izard_df_train[labels].sum()

anger       6386
disgust     2224
fear        1996
joy         5929
sadness     5260
surprise    2232
neutral     6775
dtype: int64

In [92]:
# Посмотрим на распределение эмоций в izard_df_test
izard_df_test[labels].sum()

anger       680
disgust     210
fear        217
joy         758
sadness     554
surprise    260
neutral     793
dtype: int64

In [93]:
# Посмотрим на распределение эмоций в izard_df_val
izard_df_val[labels].sum()

anger       792
disgust     282
fear        229
joy         697
sadness     679
surprise    257
neutral     777
dtype: int64

In [94]:
# Соединим все датасеты в один
emotions_df = pd.concat([goemotions_df_train, izard_df_train, izard_df_test, izard_df_val])

In [95]:
emotions_df.shape

(114384, 8)

In [101]:
emotions_df[labels].sum()

anger       15942
disgust      8017
fear         5639
joy         15367
sadness     13251
surprise     8263
neutral     63643
dtype: int64

In [102]:
# Разобьем emotions_df на train, test и validation
train_size = int(0.6 * len(emotions_df))
valid_size = int(0.2 * len(emotions_df))
test_size = len(emotions_df) - train_size - valid_size
print(train_size, valid_size, test_size)

68630 22876 22878


In [103]:
gotrain_df = emotions_df.sample(train_size)
emotions_df = emotions_df.drop(gotrain_df.index)
govalid_df = emotions_df.sample(valid_size)
gotest_df = emotions_df.drop(govalid_df.index)

In [104]:
print(gotrain_df.shape)
print(govalid_df.shape)
print(gotest_df.shape)

(68630, 8)
(22876, 8)
(15962, 8)


In [105]:
# Соединим датасеты в один
crytrain_df = pd.read_csv("content/train.csv")
cryvalid_df = pd.read_csv("content/valid.csv")

In [106]:
train_df = pd.concat([crytrain_df, gotrain_df])
valid_df = pd.concat([cryvalid_df, govalid_df])
test_df = gotest_df

print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(112040, 8)
(28302, 8)
(15962, 8)


In [107]:
# Создадим датасеты для Hugging Face
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)
test_dataset = Dataset.from_pandas(test_df)

In [108]:
data = DatasetDict({"train": train_dataset, "validation": valid_dataset, "test": test_dataset})

In [109]:
data

DatasetDict({
    train: Dataset({
        features: ['text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral', '__index_level_0__'],
        num_rows: 112040
    })
    validation: Dataset({
        features: ['text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral', '__index_level_0__'],
        num_rows: 28302
    })
    test: Dataset({
        features: ['text', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral', '__index_level_0__'],
        num_rows: 15962
    })
})

In [115]:
# Сохраним датасет в формате csv
train_df.to_csv("train.csv", index=False)
valid_df.to_csv("validation.csv", index=False)
test_df.to_csv("test.csv", index=False)